<h1>Installs</h1>
- This file prepares the data such that we can upload it to opensearch (So that we can have vecotrs and metadata all in one DB). We use the same e5 model as in strategy_1 but chunk the data with the SentenceTransformersTokenTextSplitter method. 

In [1]:
! pip install pinecone-client
! pip install faunadb
! pip install ndg-httpsclient
! pip install pyopenssl
! pip install pyasn1
! pip install -U sentence-transformers
! pip install transformers

   ---------------------------------------- 0.0/132.8 kB ? eta -:--:--
   ---------------------------------------- 132.8/132.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2



<h1>Extract Data from .csv</h1>
*   AB = Column that corresponds to the text of the abstract


In [2]:
import csv
from itertools import islice
import uuid

# Load data, store abstract text
batch_input = []
with open('../../INLPT_data/articles.csv', encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append([row["AB"],row["PMID"],row["TI"],row["PB"],row["FAU"],row["FED"],row["DP"],row["OTO"], row["OT"],row["OWN"], row["LR"], row["JT"], row["MH"],row["ISBN"]])

In [3]:
print(len(batch_input))

57560



<h1>Embedd Data from .csv</h1>
1. DIvide each abstract into chunks such that each chunks text will be tokenized to max 512 Tokens


In [4]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
import re
import tqdm

c:\Users\HRida\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#embed data
# model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
# tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
# max_token_size = 350
# print(model)
# print(tokenizer)

model_name = 'intfloat/e5-base-v2'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_token_size = 512
print(model)
print(tokenizer)


No sentence-transformers model found with name intfloat/e5-base-v2. Creating a new one with MEAN pooling.
config.json: 100%|██████████| 650/650 [00:00<00:00, 649kB/s]
c:\Users\HRida\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HRida\.cache\huggingface\hub\models--intfloat--e5-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)
BertTokenizerFast(name_or_path='intfloat/e5-base-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=Fa

In [6]:
splitter = SentenceTransformersTokenTextSplitter(
    model_name=model_name,  
    chunk_overlap=20,  
)
print(splitter.maximum_tokens_per_chunk)

modules.json: 100%|██████████| 387/387 [00:00<00:00, 386kB/s]
README.md: 100%|██████████| 67.6k/67.6k [00:00<00:00, 834kB/s]


sentence_bert_config.json: 100%|██████████| 57.0/57.0 [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 200/200 [00:00<?, ?B/s] 

512


In [7]:
chunked_data = []

#Chunk abstract text, prepare for emebdding
for abstract_text in batch_input:
    chunked_data.append(splitter.split_text(text=abstract_text[0]))

In [8]:
print(len(chunked_data))

57560


In [9]:
chunks = []

#flatten data and append metadata to each chunk -> list of [chunk, PMID, TI, PB, FAU, FED,DP,OTO,ISBN]
for id, abstract in enumerate(chunked_data):
    for chunk in abstract:
        chunks.append([chunk, batch_input[id][1],batch_input[id][2],batch_input[id][3],batch_input[id][4],batch_input[id][5],batch_input[id][6],batch_input[id][7],batch_input[id][8],batch_input[id][9],batch_input[id][10],batch_input[id][11],batch_input[id][12]])

In [11]:

chunks[0]

['clinical characteristics : cask disorders include a spectrum of phenotypes in both females and males. two main types of clinical presentation are seen : microcephalywith pontine and cerebellar hypoplasia ( micpch ), generally associated withpathogenic loss - of - function variants in cask. x - linked intellectual disability ( xlid ) with or without nystagmus, generally associated with hypomorphic caskpathogenic variants. micpch is typically seen in females with moderate - to - severeintellectual disability, progressive microcephaly with or without ophthalmologicanomalies, and sensorineural hearing loss. most are able to sit independently ; 20 % - 25 % attain the ability to walk ; language is nearly absent in most. neurologicfeatures may include axial hypotonia, hypertonia / spasticity of the extremities, and dystonia or other movement disorders. nearly 40 % have seizures by age tenyears. behaviors may include sleep disturbances, hand stereotypies, and selfbiting. micpch in males may 

In [15]:
len(chunks[0][0])

2293

In [21]:
lengths = [len(tokenizer.encode(chunk[0])) for chunk in chunks]
print(min(lengths), max(lengths))

4 517


In [10]:
print(len(chunks))

62333


In [ ]:
#add embedding vector of chunk to the list
for chunk in tqdm(chunks):
    chunk.insert(0, model.encode(chunk[0]).tolist())

In [ ]:
import pickle

#save List in File. This file will be later accesssed and the data will be uploaded to open search

path = "./data/data.txt"
with open(path, 'wb') as file:
    pickle.dump(chunks, file)